<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/MENADATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [364]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
dataG = pd.read_csv('/content/drive/My Drive/MENA/golbal.csv',encoding = "ISO-8859-1", engine = 'python', usecols=[1,2,3,8,10,19,20,21,22,26,27,29,35,41,71,84,100,101,103,58])
dataLabels=np.array(dataG.columns)

MENA=['Algeria', 'Bahrain', 'Djibouti',' Egypt', 'Iran', 'Iraq',  'Jordan','Kuwait', 'Lebanon',' Libya', 'Malta',' Morocco', 'Oman',' Qatar', 'Saudi Arabia', 'Syria', 'Tunisia', 'United Arab Emirates', 'Palestine', 'Yemen','Ethiopia' ,' Sudan' ]
data2015 = pd.read_csv('/content/drive/My Drive/MENA/2015.csv',encoding = "ISO-8859-1", engine = 'python')
dataLabels2015=np.array(data2015.columns)
data2016 = pd.read_csv('/content/drive/My Drive/MENA/2016.csv',encoding = "ISO-8859-1", engine = 'python')
dataLabels2016=np.array(data2016.columns)
data2017 = pd.read_csv('/content/drive/My Drive/MENA/2017.csv',encoding = "ISO-8859-1", engine = 'python')
dataLabels2017=np.array(data2017.columns)
data20152016=data2016.append(data2015)
yearsData=data20152016.append(data2017)
data=yearsData.append(dataG)


In [373]:
data.columns

Index(['Country', 'Dystopia Residual', 'Dystopia.Residual',
       'Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'Family',
       'Freedom', 'Generosity', 'Happiness Rank', 'Happiness Score',
       'Happiness.Rank', 'Happiness.Score', 'Health (Life Expectancy)',
       'Health..Life.Expectancy.', 'Lower Confidence Interval', 'Region',
       'Standard Error', 'Trust (Government Corruption)',
       'Trust..Government.Corruption.', 'Upper Confidence Interval',
       'Whisker.high', 'Whisker.low', 'attacktype1_txt', 'claimed',
       'country_txt', 'crit1', 'crit2', 'crit3', 'doubtterr', 'gname', 'iday',
       'imonth', 'iyear', 'natlty1_txt', 'nkillter', 'nwound', 'nwoundte',
       'region_txt', 'success', 'suicide', 'targtype1_txt',
       'weapsubtype1_txt'],
      dtype='object')

In [369]:
data.shape
data_C=data.shape[1]
data_R=data.shape[0]
print(data_C)
print(data_R)

42
182161


In [0]:
data_C=data.shape[1]
data_R=data.shape[0]
for c in range(0,data_C+1):
    Region_nums=[]
    Region_nums=data[dataLabels[c]].unique()
    Region_nums={Region_nums[i]: i for i in range(len(Region_nums))}
    Regions = Region_nums
    data[dataLabels[c]].replace(Regions, inplace=True)


For computational reasons, the best subset cannot be applied for any large n due to the 2n complexity. Forward Stepwise begins with a model containing no predictors, and then adds predictors to the model, one at the time. At each step, the variable that gives the greatest additional improvement to the fit is added to the model.

In [0]:
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

In [0]:
#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook
import itertools
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

#Initialization variables
X =data
X = X[['Country', 'Dystopia Residual', 'Dystopia.Residual',
       'Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'Family',
       'Freedom', 'Generosity', 'Happiness Rank', 'Happiness Score',
       'Happiness.Rank', 'Happiness.Score', 'Health (Life Expectancy)',
       'Health..Life.Expectancy.', 'Lower Confidence Interval', 'Region',
       'Standard Error', 'Trust (Government Corruption)',
       'Trust..Government.Corruption.', 'Upper Confidence Interval',
       'Whisker.high', 'Whisker.low', 'attacktype1_txt', 'claimed',
       'country_txt', 'crit1', 'crit2', 'crit3', 'doubtterr', 'gname', 'iday',
       'imonth', 'iyear', 'natlty1_txt', 'nkillter', 'nwound', 'nwoundte',
       'region_txt', 'success', 'suicide', 'targtype1_txt',
       'weapsubtype1_txt']]
Y = X.pop('gname')
k = 10
remaining_features = list(X.columns.values)
features = []
RSS_list, R_squared_list = [np.inf], [np.inf] #Due to 1 indexing of the loop...
features_list = dict()

for i in range(1,k+1):
    best_RSS = np.inf
    
    for combo in itertools.combinations(remaining_features,1):

            RSS = fit_linear_reg(X[list(combo) + features],Y)   #Store temp result 

            if RSS[0] < best_RSS:
                best_RSS = RSS[0]
                best_R_squared = RSS[1] 
                best_feature = combo[0]

    #Updating variables for next loop
    features.append(best_feature)
    remaining_features.remove(best_feature)
    
    #Saving values for plotting
    RSS_list.append(best_RSS)
    R_squared_list.append(best_R_squared)
    features_list[i] = features.copy()

In [0]:
df1 = pd.concat([pd.DataFrame({'features':features_list}),pd.DataFrame({'RSS':RSS_list, 'R_squared': R_squared_list})], axis=1, join='inner')
df1['numb_features'] = df1.index